In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import random
import csv
import json
import pandas as pd
print("Libraries Imported")

Libraries Imported


In [3]:
# Spoofer code
try:

    import sys
    import os

    from fp.fp import FreeProxy
    from fake_useragent import UserAgent
    from bs4 import BeautifulSoup
    from selenium import webdriver
    from selenium.webdriver import Chrome
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.common.action_chains import ActionChains
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.common.exceptions import TimeoutException
    import time
    print('all module are loaded ')

except Exception as e:

    print("Error ->>>: {} ".format(e))


class Spoofer(object):

    def __init__(self, country_id=['US', 'HK'], rand=True, anonym=True):
        self.country_id = country_id
        self.rand = rand
        self.anonym = anonym
        self.userAgent, self.ip = self.get()

    def get(self):
        ua = UserAgent()
        proxy = FreeProxy(country_id=self.country_id, rand=self.rand, anonym=self.anonym).get()
        ip = proxy.split("://")[1]
        return ua.random, ip


class DriverOptions(object):

    def __init__(self):

        self.options = Options()
        self.options.add_argument('--no-sandbox')
        self.options.add_argument('--start-maximized')
        self.options.add_argument('--start-fullscreen')
        self.options.add_argument('--single-process')
        self.options.add_argument('--disable-dev-shm-usage')
        self.options.add_argument("--incognito")
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        self.options.add_experimental_option('useAutomationExtension', False)
        self.options.add_experimental_option("excludeSwitches", ["enable-automation"])
        self.options.add_argument("disable-infobars")

        self.helperSpoofer = Spoofer()

        self.options.add_argument('user-agent={}'.format(self.helperSpoofer.userAgent))
        self.options.add_argument('--proxy-server=%s' % self.helperSpoofer.ip)


class WebDriver(DriverOptions):

    def __init__(self, path=''):
        DriverOptions.__init__(self)
        self.driver_instance = self.get_driver()

    def get_driver(self):

        print("""
        IP:{}
        UserAgent: {}
        """.format(self.helperSpoofer.ip, self.helperSpoofer.userAgent))

        PROXY = self.helperSpoofer.ip
        webdriver.DesiredCapabilities.CHROME['proxy'] = {
            "httpProxy":PROXY,
            "ftpProxy":PROXY,
            "sslProxy":PROXY,
            "noProxy":None,
            "proxyType":"MANUAL",
            "autodetect":False
        }
        webdriver.DesiredCapabilities.CHROME['acceptSslCerts'] = True

        path = os.path.join(os.getcwd(), '/chromedriver.exe')

        driver = webdriver.Chrome()
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
            "source":
                "const newProto = navigator.__proto__;"
                "delete newProto.webdriver;"
                "navigator.__proto__ = newProto;"
        })

        return driver

all module are loaded 


In [4]:
def create_csv(fields, rows, dirname, filename):
    with open(dirname+filename, 'w', encoding='UTF8', newline='') as csvfile: 
        # creating a csv writer object
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(fields) 
        csvwriter.writerows(rows)
    print(f"Saved csv with name {filename}")

In [5]:
# Create anti bot detection web driver

while True:
    try:
        driver = WebDriver()
        break
    except: # If Driver creation fails try again
        print("Driver creation failed.....Trying again")
        continue
print("Driver Successfully created")


        IP:103.73.194.2:80
        UserAgent: Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.15 (KHTML, like Gecko) Chrome/24.0.1295.0 Safari/537.15
        
Driver Successfully created


In [11]:
# Create Driver instance and open the webpage through selenium

driverinstance = driver.driver_instance
url = "https://www.expedia.com/Hotel-Search?adults=2&d1=2022-02-07&d2=2022-02-08&destination=Jaipur%2C%20Rajasthan%2C%20India&endDate=2022-02-08&latLong=26.891643000499514%2C75.74652282695956&regionId=1669&rooms=1&semdtl=&sort=RECOMMENDED&startDate=2022-02-07&theme=&useRewards=false&userIntent="
driverinstance.get(url)

# Click on the "SHOW MORE" button 20 times to load enough hotels 
for i in range(1):
    sleep_time = 5 + 5*random.random()
    time.sleep(sleep_time)
    driverinstance.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    try:
        button = driverinstance.find_element_by_xpath('//*[@class="uitk-button uitk-button-medium uitk-button-secondary"]')
        button.click()
    except:
        print("Button not Found, ending search")
        break

# Create BeautifulSoup instance and search for each hotel on the page to get it's link

soup = str(BeautifulSoup(driverinstance.page_source, 'html.parser'))
soup = BeautifulSoup(soup, 'lxml')

ol = soup.find('ol' , class_  = 'results-list no-bullet').find_all('li', attrs={"data-stid":"property-listing"})

hotel_url_li = []
hotel_name_li = []
hotel_price_li = []

for li in ol:
    hotel_url = li.find('a', class_='listing__link uitk-card-link')
    hotel_name = li.find('h3' , class_='uitk-heading-5 truncate-lines-2 all-b-padding-half pwa-theme--grey-900 uitk-type-heading-500')
    hotel_price_div = li.find('div', attrs= {"data-stid": "content-hotel-price"})
    if "$" not in hotel_price_div.text:
        hotel_price = ""
    else:
        hotel_price = li.find('span', class_ = "uitk-cell loyalty-display-price all-cell-shrink", attrs= {"data-stid" : "price-lockup-text"})
    hotel_url_li.append(hotel_url)
    hotel_name_li.append(hotel_name)
    hotel_price_li.append(hotel_price)

AttributeError: 'NoneType' object has no attribute 'text'

In [8]:
print(len(ol))

300


In [10]:
hotel_url_list = []
hotel_info = []

hotel_length = len(hotel_url_li)
print(f"Number of hotels found: {hotel_length}\n")
for i in range(hotel_length):
    hotel_url_list.append("https://www.expedia.com"+hotel_url_li[i].attrs['href'])
    ls = [i+1, hotel_name_li[i].text, hotel_price_li[i].text[1:]]
    hotel_info.append(ls)

Number of hotels found: 50



In [11]:
# Create CSV for name and price of the hotels

fields = ['hotel_id', 'hotel_name', 'hotel_price']
create_csv(fields, hotel_info, "Project-Dataset/", "jaipur_hotels.csv")

Saved csv with name jaipur_hotels.csv


In [9]:
# Save all the links in a text file

with open("india_hotel_urls.txt", 'w') as output:
    for row in hotel_url_list:
        output.write(str(row) + '\n')